In [13]:
import cv2
import numpy as np
import tensorflow as tf
import os

import subprocess#start process
import pyautogui#mouse movement

from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image


In [14]:
cap = cv2.VideoCapture(0) # default beepített webkamera használata
model = keras.models.load_model("hand_gesture_classification6.h5")

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (0,50)
fontScale              = 1
fontColor              = (0,255,0)
lineType               = 2




In [15]:
DEBUG_MODE = False
ACCESIBILITY_MODE = False

In [16]:
def call_app(ss,accesibility_mode):
    if ss == 1: #five
        if accesibility_mode:
            pyautogui.moveRel(0, 10)# move mouse 10 pixels down
        else:
            subprocess.Popen("C:\\Program Files (x86)\\Notepad++\\notepad++.exe")
       
    elif ss == 2:#two
        if accesibility_mode:
            pyautogui.moveRel(0, -10)  # move mouse 10 pixels up

        else:
            subprocess.Popen("C:\\Users\\Seres\\AppData\\Local\\Microsoft\\Teams\\current\\Teams.exe")

        
    elif ss == 3:#zero
        if accesibility_mode:
            pyautogui.moveRel(10, 0)# move mouse 10 pixels right

        else:
            subprocess.Popen("control")  # move mouse 10 pixels down

        
        


In [17]:
 #Image processing
 #default beepített webkamera használata
def get_feed():
    while(1):
        try:
            ret, frame = cap.read()
            frame = cv2.flip(frame,1)
            #adott regio ahol a kezet kell tartani
            roi = frame[100:300,100:300]
            #cv2.rectangle(image, start_point, end_point, color, thickness)
            cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0) #zöld négyzetet rajzolunk arra a helyre ahova  kezet kerjuk

            #HSV színskála használata
            hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

            #a bőr színeinek szinmegfelelő hatarai, update majd
            l_skin = np.array([0,20,70],dtype=np.uint8)
            h_skin = np.array([20,255,255],dtype=np.uint8)

            #adott bőr régio maszkolása
            mask = cv2.inRange(hsv,l_skin,h_skin)

            kernel = np.ones((3,3),np.uint8)
            #a mask dilatációja, hogy ne legyenek fekete pontok
            #3x3-as szűrőt használunk a dilatációra, négyszer iterálunk
            mask = cv2.dilate(mask,kernel,iterations = 4)
            #Gauss szűrővel pedig blureolunk a képen, hogy ne legyenek élesek a határok
            mask = cv2.GaussianBlur(mask,(5,5),100)
            
            c = mask
            c = np.expand_dims(c, axis = 0)
            c = c.reshape(1, 200, 200, 1)
            c =c/255


            value = model.predict(c)
            
            global ACCESIBILITY_MODE
            if value[0][0] > 0.90:
                    cv2.putText(frame,"Five: "+str(value[0][0]), 
                                                bottomLeftCornerOfText, 
                                                font, 
                                                fontScale,
                                                fontColor,
                                                lineType)
                    call_app(1,ACCESIBILITY_MODE)

            elif value[0][1] > 0.90:
                    cv2.putText(frame,"Nothing "+str(value[0][1]), 
                                                bottomLeftCornerOfText, 
                                                font, 
                                                fontScale,
                                                fontColor,
                                                lineType)
                    

            elif value[0][2] > 0.90:
                    cv2.putText(frame,"Two  "+str(value[0][2]), 
                                                bottomLeftCornerOfText, 
                                                font, 
                                                fontScale,
                                                fontColor,
                                                lineType)

                    call_app(2,ACCESIBILITY_MODE)
            elif value[0][3] > 0.90:
                    cv2.putText(frame,"Zero "+str(value[0][3]), 
                                                bottomLeftCornerOfText, 
                                                font, 
                                                fontScale,
                                                fontColor,
                                                lineType)

                    call_app(3,ACCESIBILITY_MODE)
            global DEBUG_MODE
            if DEBUG_MODE is True:
                cv2.putText(frame,str(value), 
                                                    (0,450), 
                                                    font, 
                                                    0.5,
                                                    (0,0,255),
                                                    lineType)

            
            
            
            
            cv2.imshow('mask',mask)
            cv2.imshow('frame',frame)


        except:
            pass
        k = cv2.waitKey(5) & 0xFF
        if k == 50:#kettest nyom
            DEBUG_MODE = not DEBUG_MODE
        if k == 51:#hármast nyom
            ACCESIBILITY_MODE = not ACCESIBILITY_MODE
            print(ACCESIBILITY_MODE)
        if k==27:
            break;
        
    cv2.destroyAllWindows()
    cap.release()

        
        

In [18]:
get_feed()

True
